### Restart and Run All Cells

In [1]:
import glob
import os
import pandas as pd
from datetime import date, timedelta
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','amount':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}'}

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"
dts_path = "/Users/User/Downloads/Datasets/"

# Directory to be monitored
directory = '/Users/User/Downloads'
# Dictionary of file categories and their extensions
categories = {
    'Images': ['jpeg', 'jpg', 'png'],
    'PDFs': ['pdf'],
    'Datasets': ['csv', 'xlsx', 'xls', 'json'],
    'Videos': ['mp4','m4a'],
    'Zips': ['zip'],
    'EXEs': ['exe', 'msi'],
    'Pythons': ['ipynb'],
}
pd.options.display.max_rows = 11
pd.options.display.float_format = '{:.2f}'.format
year = 2023
month = 8
today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 8, 25), datetime.date(2023, 8, 24))

In [2]:
num_business_days = BDay(2)
yesterday = today - num_business_days
print(f'today: {today}')
yesterday = yesterday.date()
print(f'yesterday: {yesterday}')

today: 2023-08-25
yesterday: 2023-08-23


## Must rerun whenever there is change in orders

In [3]:
sql = """
SELECT trade, name, qty, price AS target, active, reason, market, xdate 
FROM orders
ORDER BY trade, name"""
orders = pd.read_sql(sql, conlite)
orders.shape

(48, 8)

In [4]:
file_name = "Price-Today.csv"
input_file = data_path + file_name
input_file

'../data/Price-Today.csv'

In [5]:
prices = pd.read_csv(input_file)
prices.shape

(190, 10)

In [6]:
df_merge  = pd.merge(orders, prices, on = 'name', how='inner')
df_merge.shape

(48, 17)

In [7]:
df_merge['spd'] = 0

In [8]:
colo = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'price', 'change', '%change',
        'active', 'xdate']
df_merge[colo].shape

(48, 12)

In [9]:
file_name = "Price-for-Order.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

df_merge[colo].to_csv(output_file, header=True, index=False)
df_merge[colo].to_csv(osd_file, header=True, index=False)

### call ruby ruby\order-log-new.rb

In [10]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [11]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [12]:
!ruby ruby\\order-log-new.rb

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
B,AH,-16,RD15pct,SET,600,31.0,35.0,0.75,2.189781,0,2023-08-29
B,AWC,-22,RD25pct,SET50,4000,3.66,4.1,-0.02,-0.485437,0,2022-12-22
B,BBL,-25,XXX,SET50,400,155.0,167.5,1.0,0.600601,0,2023-09-08
B,BEM,-25,XXX,SET50,2500,7.8,9.05,-0.05,-0.549451,0,2022-12-22
B,BH,-8,10S,SET50,100,250.0,258.0,-2.0,-0.769231,1,2022-12-22
B,CK,-12,XXX,SET100,1000,22.1,23.3,0.0,0.0,0,2022-12-22
B,CPN,-18,10S,SET50,300,64.5,69.0,-0.5,-0.719424,0,2022-12-22
B,DELTA,-55,XXX,SET50,200,90.0,107.5,-3.0,-2.714932,0,2022-12-22
B,DIF,-13,RD40pct,SET,3000,8.65,9.3,0.15,1.639344,1,2023-08-15
B,HANA,-15,10S,SET100,400,59.0,62.75,-0.25,-0.396825,0,2022-12-22
B,ICHI,-24,XXX,SET,1500,14.3,16.7,0.0,0.0,0,2022-12-22
B,III,-11,10S,SET,2000,10.8,11.9,-0.1,-0.833333,0,2022-12-22
B,IVL,-8,RD30pct,SET50,800,27.5,29.5,0.75,2.608696,1,2023-08-28
B,JASIF,-9,RD35pct,SET,4000,6.5,6.95,0.1,1.459854,1,2023-08-21
B,JMART,-34,RD45pct,SET50,3000,18.9,22.3,1.1,5.188679

In [13]:
os.chdir("C:\\users\\user\\onedrive\\a7\\daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\daily'

In [14]:
file_name   = 'orders-log.csv'
input_file = data_path + file_name

df = pd.read_csv(input_file, sep=',', index_col=None)
df['amount'] = df['qty'] * df['target']
df.query('spd >= -3 and spd <= 3')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
18,B,MCS,-3,RD60pct,SET,4000,6.00,6.15,0.10,1.65,1,2023-03-17,24000.00
20,B,ORI,-3,RD05pct,SET100,5000,10.50,10.80,0.20,1.89,1,2023-08-28,52500.00
32,B,TFFIF,-2,DOS,SET,10000,7.35,7.45,-0.05,-0.67,1,2023-08-31,73500.00
36,S,AIMIRT,1,DOS,SET,10000,11.10,11.00,0.10,0.92,1,2023-08-23,111000.00
41,S,GVREIT,1,DOS,SET,6000,7.40,7.35,0.00,0.00,2,2023-08-28,44400.00
47,S,WHART,2,DOS,SET,10000,10.60,10.40,0.00,0.00,2,2023-08-21,106000.00


In [15]:
df.query('active == 2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
41,S,GVREIT,1,DOS,SET,6000,7.40,7.35,0.00,0.00,2,2023-08-28,44400.00
47,S,WHART,2,DOS,SET,10000,10.60,10.40,0.00,0.00,2,2023-08-21,106000.00


In [16]:
df.query('active == 1')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
4,B,BH,-8,10S,SET50,100,250.00,258.00,-2.00,-0.77,1,2022-12-22,25000.00
8,B,DIF,-13,RD40pct,SET,3000,8.65,9.30,0.15,1.64,1,2023-08-15,25950.00
12,B,IVL,-8,RD30pct,SET50,800,27.50,29.50,0.75,2.61,1,2023-08-28,22000.00
13,B,JASIF,-9,RD35pct,SET,4000,6.50,6.95,0.10,1.46,1,2023-08-21,26000.00
14,B,JMART,-34,RD45pct,SET50,3000,18.90,22.30,1.10,5.19,1,2023-08-24,56700.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,B,QH,-9,10S,SET100,10000,2.10,2.28,0.00,0.00,1,2023-08-23,21000.00
30,B,STA,-19,RD70pct,SET100,2500,12.70,14.60,0.70,5.04,1,2023-08-24,31750.00
32,B,TFFIF,-2,DOS,SET,10000,7.35,7.45,-0.05,-0.67,1,2023-08-31,73500.00
36,S,AIMIRT,1,DOS,SET,10000,11.10,11.00,0.10,0.92,1,2023-08-23,111000.00


In [17]:
pd.options.display.max_rows = 35
df.query('active == 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
0,B,AH,-16,RD15pct,SET,600,31.00,35.00,0.75,2.19,0,2023-08-29,18600.00
1,B,AWC,-22,RD25pct,SET50,4000,3.66,4.10,-0.02,-0.49,0,2022-12-22,14640.00
2,B,BBL,-25,XXX,SET50,400,155.00,167.50,1.00,0.60,0,2023-09-08,62000.00
3,B,BEM,-25,XXX,SET50,2500,7.80,9.05,-0.05,-0.55,0,2022-12-22,19500.00
5,B,CK,-12,XXX,SET100,1000,22.10,23.30,0.00,0.00,0,2022-12-22,22100.00
6,B,CPN,-18,10S,SET50,300,64.50,69.00,-0.50,-0.72,0,2022-12-22,19350.00
7,B,DELTA,-55,XXX,SET50,200,90.00,107.50,-3.00,-2.71,0,2022-12-22,18000.00
9,B,HANA,-15,10S,SET100,400,59.00,62.75,-0.25,-0.40,0,2022-12-22,23600.00
10,B,ICHI,-24,XXX,SET,1500,14.30,16.70,0.00,0.00,0,2022-12-22,21450.00
11,B,III,-11,10S,SET,2000,10.80,11.90,-0.10,-0.83,0,2022-12-22,21600.00


In [18]:
df.query('spd <= -40 and trade == "B"')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
7,B,DELTA,-55,XXX,SET50,200,90.00,107.50,-3.00,-2.71,0,2022-12-22,18000.00
22,B,PTTEP,-49,XXX,SET50,200,134.50,159.00,1.50,0.95,0,2023-08-15,26900.00


In [19]:
df.query('trade == "B" & spd > 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [20]:
colx = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'current', 'change', '%change',
        'active', 'xdate']

file_name = "orders-log.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

df[colx].to_csv(output_file, header=True, index=False)
df[colx].to_csv(osd_file, header=True, index=False)

### Convert column name to make it valid for df.query

In [21]:
df['percent'] = df['%change']

In [22]:
df.query('percent < -2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
7,B,DELTA,-55,XXX,SET50,200,90.00,107.50,-3.00,-2.71,0,2022-12-22,18000.00,-2.71
21,B,PSH,-8,10S,SET,2000,12.20,13.00,-0.40,-2.99,1,2023-03-10,24400.00,-2.99
25,B,SAPPE,-14,10S,SET,300,89.00,92.50,-3.25,-3.39,0,2022-12-22,26700.00,-3.39


In [23]:
df.query('percent >= 2.00')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
0,B,AH,-16,RD15pct,SET,600,31.00,35.00,0.75,2.19,0,2023-08-29,18600.00,2.19
12,B,IVL,-8,RD30pct,SET50,800,27.50,29.50,0.75,2.61,1,2023-08-28,22000.00,2.61
14,B,JMART,-34,RD45pct,SET50,3000,18.90,22.30,1.10,5.19,1,2023-08-24,56700.00,5.19
15,B,JMT,-27,RD30pct,SET50,1800,39.75,46.50,1.00,2.20,0,2023-08-24,71550.00,2.20
23,B,PTTGC,-14,RD4,SET50,600,34.50,38.00,1.50,4.11,1,2023-02-27,20700.00,4.11
28,B,SINGER,-23,RD60pct,SET100,3000,9.50,11.30,1.00,9.71,0,2023-08-03,28500.00,9.71
30,B,STA,-19,RD70pct,SET100,2500,12.70,14.60,0.70,5.04,1,2023-08-24,31750.00,5.04
31,B,SYNEX,-32,RD65pct,SET,2000,10.00,13.20,0.40,3.12,0,2023-08-25,20000.00,3.12
35,B,TTB,-12,XXX,SET50,15000,1.64,1.76,0.04,2.33,0,2022-12-22,24600.00,2.33


In [24]:
df.loc[df.trade == 'B'].sort_values('spd',ascending=False)

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
32,B,TFFIF,-2,DOS,SET,10000,7.35,7.45,-0.05,-0.67,1,2023-08-31,73500.00,-0.67
18,B,MCS,-3,RD60pct,SET,4000,6.00,6.15,0.10,1.65,1,2023-03-17,24000.00,1.65
20,B,ORI,-3,RD05pct,SET100,5000,10.50,10.80,0.20,1.89,1,2023-08-28,52500.00,1.89
16,B,KTB,-5,10S,SET50,1000,18.80,19.30,0.20,1.05,1,2023-04-20,18800.00,1.05
4,B,BH,-8,10S,SET50,100,250.00,258.00,-2.00,-0.77,1,2022-12-22,25000.00,-0.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,B,JMT,-27,RD30pct,SET50,1800,39.75,46.50,1.00,2.20,0,2023-08-24,71550.00,2.20
31,B,SYNEX,-32,RD65pct,SET,2000,10.00,13.20,0.40,3.12,0,2023-08-25,20000.00,3.12
14,B,JMART,-34,RD45pct,SET50,3000,18.90,22.30,1.10,5.19,1,2023-08-24,56700.00,5.19
22,B,PTTEP,-49,XXX,SET50,200,134.50,159.00,1.50,0.95,0,2023-08-15,26900.00,0.95


### Sell recently purchased stocks if profitable

In [25]:
pd.set_option('display.max_rows', 40)
sql = """SELECT name, qty, price AS cost, date 
FROM buys B 
JOIN stocks S 
ON stock_id = S.id 
WHERE status = 'Active'
ORDER BY name, date"""
buys = pd.read_sql(sql, conpf)
buys.shape

(108, 4)

In [26]:
colu = 'name date_x qty cost price difa %change %profit'.split()
buys_price = pd.merge(buys, prices, on='name', how='inner')
buys_price['diff'] = buys_price['price'] - buys_price['cost']
buys_price['difa'] = buys_price['diff'] * buys_price['qty']
buys_price['%profit'] = round(buys_price['difa'] / (buys_price['qty'] * buys_price['cost']) * 100,2)

In [27]:
laggards = buys_price.loc[buys_price.cost > buys_price.price,colu]
laggards.shape

(104, 8)

In [28]:
watch_list = buys_price.loc[buys_price.cost <= buys_price.price,colu]
watch_list.sort_values('name',ascending='False')

,name,date_x,qty,cost,price,difa,%change,%profit
1,AIMIRT,2023-08-17,10000,11.00,11.00,0.00,0.92,0.00
22,GVREIT,2023-08-18,3000,7.30,7.35,150.00,0.00,0.68
23,GVREIT,2023-08-25,6000,7.35,7.35,0.00,0.00,0.00
95,TFFIF,2023-07-25,10000,7.45,7.45,0.00,-0.67,0.00


### Profit per Day

In [29]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s AND MONTH(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % (year, month)
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2023 AND MONTH(sells.date) = 8
ORDER BY sells.date DESC, name


In [30]:
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days
0,JMART,2023,8,20.60,19.60,1.00,"1,200","24,720.00","23,520.00","1,200.00",5.10,126
1,ICHI,2023,8,15.90,14.30,1.60,"1,400","22,260.00","20,020.00","2,240.00",11.19,59
2,TTB,2023,8,1.76,1.64,0.12,"15,000","26,400.00","24,600.00","1,800.00",7.32,26
3,JMART,2023,8,18.30,17.00,1.30,"3,600","65,880.00","61,200.00","4,680.00",7.65,41
4,JMT,2023,8,43.50,39.75,3.75,"1,800","78,300.00","71,550.00","6,750.00",9.43,57


In [31]:
file_name = "monthly-sales.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

sells_df.to_csv(output_file, header=True, index=False)
sells_df.to_csv(osd_file, header=True, index=False)

In [32]:
sells_df['profit_per_day'] = (sells_df['gross'] / sells_df['buy_amt']) / sells_df['days'] * 100
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days,profit_per_day
0,JMART,2023,8,20.60,19.60,1.00,"1,200","24,720.00","23,520.00","1,200.00",5.10,126,0.040492
1,ICHI,2023,8,15.90,14.30,1.60,"1,400","22,260.00","20,020.00","2,240.00",11.19,59,0.189641
2,TTB,2023,8,1.76,1.64,0.12,"15,000","26,400.00","24,600.00","1,800.00",7.32,26,0.281426
3,JMART,2023,8,18.30,17.00,1.30,"3,600","65,880.00","61,200.00","4,680.00",7.65,41,0.186514
4,JMT,2023,8,43.50,39.75,3.75,"1,800","78,300.00","71,550.00","6,750.00",9.43,57,0.165508


In [33]:
average_profit_percent = sells_df['profit_per_day'].mean()
print(f"Average Profit Percent: {average_profit_percent:.2f}%")

Average Profit Percent: 0.17%
